# 🧩 **02 — Feature Engineering**

**Notebook Purpose:**
Transform insights from exploratory data analysis into *explicit, reproducible features* suitable for time-series classification. This notebook focuses on constructing lagged, rolling, and event-aware features while preserving temporal integrity and avoiding data leakage.

---

**Competition:** *Detect Reversal Points in US Equities*
**Deadline:** December 31, 2025
**Repository:** `Kaggle-Detect-Reversal-Points-in-US-Equities`
**Author:** Brice Nelson

---

**Notebook Date Created:** 2025-12-15<br>
**Notebook Last Updated:** 2025-12-15

---

## 🧭 **Goals of This Notebook**

- Ingest validated raw and baseline datasets
- Engineer time-aware features derived from Signal Descriptor columns
- Construct lag-based and rolling window features without leakage
- Encode sparse event information relevant to reversal detection
- Maintain compatibility with baseline and advanced modeling pipelines
- Persist engineered datasets to `/data/processed/`
- Document feature rationale for downstream interpretation

---

## 🔗 **Context from Prior Analysis**

Feature engineering decisions in this notebook are informed by findings from the light EDA phase, including:

- Extremely wide feature space dominated by boolean Signal Descriptor columns
- Sparse, event-based target labels (`H`, `L`, `None`)
- Strong temporal ordering within each `ticker_id`
- Need for models robust to high-dimensional, sparse inputs

Detailed exploratory analysis and visualization are deferred to
`01_eda_detailed.ipynb`.

---

## 📂 **References**

- Light EDA: `notebooks/01_eda.ipynb`
- Detailed EDA (planned): `notebooks/01_eda_detailed.ipynb`
- Project Plan: `docs/00_overview/01_reversal_points_project_plan.md`
- Feature Design Notes: `docs/01_architecture/02_feature_design.md`
- Project Structure: `docs/01_architecture/01_project_structure.md`


In [1]:
# import libraries

import os
import sys
import pandas as pd
import numpy as np
import duckdb
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

# Add project root to path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from src.data.eda_utils import get_prefix_counts

# configurations
sns.set_theme(style="darkgrid")
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 120)

plt.style.use("seaborn-v0_8")

### Load Raw Data
- Load via Duckdb
- Create a connection
- Load the training and test datasets into dataframes

In [2]:
# Create duckdb connection

conn = duckdb.connect()

# Create duckdb dataframe

train_df = conn.execute("""
                        SELECT * FROM
                            read_csv_auto('../data/raw/new_competition_data/train.csv',
                            max_line_size=5000000)""").df()
test_df = conn.execute("""
                       SELECT * FROM
                           read_csv_auto('../data/raw/new_competition_data/test.csv',
                           max_line_size=5000000)""").df()

print('Train dataframe created.')
print('Test dataframe created.')

Train dataframe created.
Test dataframe created.
